<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;
           letter-spacing:0.10px">

<p style="font-size:35px; color:white; text-align:center"> Introduction</p>
</div>

The goal of this work is to build a machine learning model that will determine from a photo whether a cat or a dog is depicted. Based on the specifics, the task is a binary classification.  
The architecture of the model will be a convolutional neural network, because it has significant advantages for working with images:
- Fewer trainable parameters, since the same set of weights is used for all neurons in the feature map - which is called the convolution core  
- A convolutional neural network receives not a vector, but a matrix or tensor at its input (unlike a fully connected neural network, or standard machine learning algorithms)

In order to speed up the training of the model, we will use the GPU. An important point is the problem of retraining in CNN, so at the design stage it is necessary to take this fact into account in advance.  
The CNN model will be built using Keras  
In the process of processing, we will make all images of a unique size of 200x200x3 (since the model must accept objects of standard sizes).  
Since there is no class imbalance (0, 1) in the dataset, Accuracy is a completely objective metric, so we will choose it as the metric

<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center"> Table of content</p>
</div>

### [**1) Library import**](#title-one)

### [**2) Getting data**](#title-two)

### [**3) EDA**](#title-three)
    
### [**4) Data preprocessing**](#title-four) 

### [**5) Creating a CNN model**](#title-five)


<a id="title-one"></a>
<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center">1) Library import</p>
</div>

In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
import sklearn
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, BatchNormalization, Dropout, Input
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from keras import callbacks
from keras.layers import Dense, Flatten, BatchNormalization, GlobalAveragePooling2D, Dropout, Input
from keras import Model
from keras.layers import  Conv2D, MaxPool2D, add
from keras.preprocessing import image
import random
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

<a id="title-three"></a>
<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center">2) Getting data</p>
</div>

A few words about the dataset:  
In the dataset, the data is already divided into train and test. For valid in the learning process, we will take part of the data from train.  
Currently, the data is in the form of images, CNN does not know how to process images in pure form, it is necessary to submit a tensor to the input, so we will deal with the representation of images in the form of a tensor.   
Let's create data generators for train, valid, test based on the dataset:

In [ ]:
training_path = '../input/cat-and-dog/training_set/training_set/'
test_path = '../input/cat-and-dog/test_set/test_set/'
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 150

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../input/cat-and-dog/training_set/training_set",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.15,
    subset='training'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../input/cat-and-dog/test_set/test_set",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=42
)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    directory="../input/cat-and-dog/training_set/training_set",
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=42,
    validation_split=0.15,
    subset='validation'
)

In [ ]:
# in order to access the generator elements, you can use the .take() method. For example:
# for batch, labels in train_ds.take(1):
#     print(batch, labels)    

<a id="title-four"></a>
<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center">3) EDA</p>
</div>

Let's see what this data is:

In [ ]:
cnt_imgs = 16  # we take 8 images for each class
cat_path = training_path + '/cats'
dog_path = training_path + '/dogs'
cat_imgs = os.listdir(cat_path)[:cnt_imgs]
dog_imgs = os.listdir(dog_path)[:cnt_imgs]

In [ ]:
counter = 0
cat_imgs_path = [cat_path + '/' + i for i in cat_imgs]
dog_imgs_path = [dog_path + '/' + j for j in dog_imgs]
all_imgs = cat_imgs_path + dog_imgs_path
random.shuffle(all_imgs)

plt.figure(figsize=(28, 10))
for img_path in all_imgs:
    plt.subplot(4, 8, counter + 1)
    img = cv2.imread(img_path)
    img = cv2.resize(img, IMAGE_SIZE)
    label = img_path[len(training_path) + 1: img_path.rfind('/')]
    plt.imshow(img)
    plt.title(label)
    plt.axis('off')
    counter += 1

Let's also see if there is an imbalance of classes:

In [ ]:
def check_cnt_label(label: str) -> int:
    """A function that should determine the number of objects of this
    class in the specified directories"""
    cnt_object = 0
    paths = [training_path, test_path]
    for path in paths:
        path += '/' + label
        cnt_object += len(os.listdir(path))
    return cnt_object

CNT_CAT = check_cnt_label('cats')
CNT_DOG = check_cnt_label('dogs')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['Cats', 'Dogs'],
    y=[CNT_CAT, CNT_DOG],
    width=[0.4, 0.4]))

fig.update_layout(title='Classes and their number in the dataset', title_x=0.5)

What can we see from this graph:
- There is no class imbalance, so the Accuracy metric is objective
- There are 10,000 objects on train, valid, test in total

<a id="title-five"></a>
<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center">4) Data preprocessing </p>
</div>

It is necessary to scale the data, because the neural network must accept numbers in the range from 0 to 1 as input. Changing the image size to the standard value (200, 200) was carried out already at the stage of creating generators. Data augmentation is not provided for this work

In [ ]:
rescale = Rescaling(scale=1.0 / 255)

train_ds = train_ds.map(lambda image, label: (rescale(image), label))

valid_ds  = valid_ds.map(lambda image, label: (rescale(image), label))

test_ds = test_ds.map(lambda image, label: (rescale(image), label))

<a id="title-six"></a>
<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center">5) Creating a CNN model</p>
</div>

To reduce overfitting, we will use the following methods:
- Change the number of epochs  
- Use callback  
- Change the complexity of the model  
- Use Dropout layers in the model

In [ ]:
# We will also set the number of epochs:
EPOCHS = 35

In [ ]:
CALLBACKS = [
    callbacks.EarlyStopping(monitor='loss', min_delta=0.01, patience=7, verbose=1),  
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, min_delta=0.01, min_lr=1e-10, patience=2, verbose=1, mode='auto')
]

In [ ]:
inputs = Input(shape=(IMAGE_SIZE + (3,)))

x = Conv2D(32, (3, 3), activation='elu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='elu')(x)
block_1_output = MaxPool2D(pool_size=(3, 3))(x)

x = Conv2D(64, (3, 3), activation='elu', padding='same')(block_1_output)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='elu', padding='same')(x)
block_2_output = add([x, block_1_output])

x = Conv2D(64, (3, 3), activation='elu', padding='same')(block_2_output)
x = BatchNormalization()(x)
x = Conv2D(64, (3, 3), activation='elu', padding='same')(x)
block_3_output = add([x, block_2_output])

x = Conv2D(128, (3, 3), activation='elu')(block_3_output)
x = MaxPool2D(pool_size=(2, 2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='elu')(x)
output = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(inputs, output)

In [ ]:
model.compile(metrics=['Accuracy'], loss='binary_crossentropy', optimizer='Adam')

In [ ]:
history = model.fit(train_ds, epochs=EPOCHS, validation_data=valid_ds, callbacks=CALLBACKS)

In [ ]:
plt.figure(figsize=(24, 8))
plt.title('Visualization of the model learning process', fontsize=14)
plt.plot(np.arange(1, EPOCHS + 1), history.history['Accuracy'], label='The fraction of correct answers on the training set') 
plt.plot(np.arange(1, EPOCHS + 1), history.history['val_Accuracy'], label='The fraction of correct answers of the validation set')
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('The fraction of correct answers', fontsize=14)
plt.xticks(np.arange(1, EPOCHS + 1), fontsize=14)
plt.yticks(fontsize=14)
plt.xlim(1, 15)
plt.grid()
plt.legend(fontsize=14)
plt.show()

In [ ]:
model.summary()

In [ ]:
test_result = model.evaluate(test_ds, verbose=0)[1]
train_result = model.evaluate(train_ds, verbose=0)[1]
print(f'Metric on test: {test_result}')
print(f'Metric on train: {train_result}')

<a id="title-six"></a>
<div style="color:white;
           display:fill;
           border-radius:15px;
           background-color:#314f90;
           font-size:200%;
           font-family:Newtime-Roman;
           font-style: Arial;">

<p style="font-size:35px; color:white; text-align:center">END</p>
</div>